# **Mitigating Bias in Binary Classification Setting**

This tutorial shows how we can use the holisticai library to implement bias mitigation strategies.

Algorithms Covered:

 - Reweighing [Preprocessing]

 - Learning Fair Representations [Preprocessing]

 - Grid Search Reduction [Inprocessing]

 - Exponentiated Gradient Reduction [Inprocessing]

 - Calibrated Equalized Odds [Postprocessing]

 - Equalized Odds [Postprocessing]

 - Reject Option Classification [Postprocessing]

## Load and preprocess Data

In [1]:
# sys path
import sys
sys.path.append('../../')

In [2]:
# Base Imports
import numpy as np
import pandas as pd

# Settings
np.random.seed(0)
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Adult dataset
from holisticai.datasets import load_adult
from sklearn.model_selection import train_test_split

In [4]:
# Dataset
data = load_adult()

# Dataframe
df = pd.concat([data["data"], data["target"]], axis=1)
protected_variables = ["sex", "race"]
output_variable = ["class"]

# Simple preprocessing
y = df[output_variable].replace({">50K": 1, "<=50K": 0})
X = pd.get_dummies(df.drop(protected_variables + output_variable, axis=1))
group = ["sex"]
group_a = df[group] == "Female"
group_b = df[group] == "Male"
data_ = [X, y, group_a, group_b]

# Train test split
dataset = train_test_split(*data_, test_size=0.2, shuffle=True)
train_data = dataset[::2]
test_data = dataset[1::2]

In [5]:
# the dataframe
data['frame']

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25.0,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
3,44.0,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K
4,18.0,NaN,103497.0,Some-college,10.0,Never-married,NaN,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27.0,Private,257302.0,Assoc-acdm,12.0,Married-civ-spouse,Tech-support,Wife,White,Female,0.0,0.0,38.0,United-States,<=50K
48838,40.0,Private,154374.0,HS-grad,9.0,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
48839,58.0,Private,151910.0,HS-grad,9.0,Widowed,Adm-clerical,Unmarried,White,Female,0.0,0.0,40.0,United-States,<=50K
48840,22.0,Private,201490.0,HS-grad,9.0,Never-married,Adm-clerical,Own-child,White,Male,0.0,0.0,20.0,United-States,<=50K


In [6]:
# efficacy metrics from sklearn
from sklearn import metrics

# dictionnary of metrics
metrics_dict={
        "Accuracy": metrics.accuracy_score,
        "Balanced accuracy": metrics.balanced_accuracy_score,
        "Precision": metrics.precision_score,
        "Recall": metrics.recall_score,
        "F1-Score": metrics.f1_score}

# efficacy metrics dataframe helper tool
def metrics_dataframe(y_pred, y_true, metrics_dict=metrics_dict):
    metric_list = [[pf, fn(y_true, y_pred)] for pf, fn in metrics_dict.items()]
    return pd.DataFrame(metric_list, columns=["Metric", "Value"]).set_index("Metric")

## Baseline

In [7]:
# sklearn imports
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# holisticai imports
from holisticai.bias.metrics import classification_bias_metrics

In [8]:
# Implement a Logistic Regression for baseline results

# train
X, y, group_a, group_b = train_data
scaler = StandardScaler()
Xt = scaler.fit_transform(X)
model = LogisticRegression()
model.fit(Xt, y)

# test
X, y, group_a, group_b = test_data
Xt = scaler.transform(X)
y_pred = model.predict(Xt)
y_proba = model.predict_proba(Xt)
y_score = y_proba[:,1]
y_true = y

In [9]:
# Baseline efficacy
metrics_dataframe(y_pred, y_true)

,Value
Metric,
Accuracy,0.850241
Balanced accuracy,0.764963
Precision,0.728822
Recall,0.600681
F1-Score,0.658576


The efficacy assessment shows we can predict salary class with accuracy (0.85). Let's take a look at bias.

In [10]:
# Baseline Bias
classification_bias_metrics(group_a, group_b, y_pred, y_true, metric_type='both')

,Value,Reference
Metric,,
Statistical Parity,-0.178353,0
Disparate Impact,0.309067,1
Four Fifths Rule,0.309067,1
Cohen D,-0.457755,0
Equality of Opportunity Difference,-0.059581,0
False Positive Rate Difference,-0.082695,0
Average Odds Difference,-0.071138,0
Accuracy Difference,0.122388,0


As expected, the results are quite biased, for instance we look at a disparate impact of 0.3. We will show how we can mitigate some of this bias without losing much efficacy.

## Preprocessing Strategies

## Reweighing
References: 

Kamiran, Faisal, and Toon Calders. "Data preprocessing techniques for classification
        without discrimination." Knowledge and information systems 33.1 (2012): 1-33.

https://towardsdatascience.com/reweighing-the-adult-dataset-to-make-it-discrimination-free-44668c9379e8

In [11]:
# import
from holisticai.bias.mitigation import Reweighing

In [12]:
# initialise
rew = Reweighing()

In [13]:
# fit reweighing object to training data
X_train, y_train, group_a, group_b = train_data
rew.fit(y_train, group_a, group_b)

In [14]:
# access the new sample_weight
sw = rew.estimator_params["sample_weight"]
sw

array([0.85374028, 1.09231797, 0.85374028, ..., 0.78794552, 0.85374028,
       0.78794552])

In [15]:
# Implement a Logistic Regression (with Reweighing)

# train (with reweighing)
X, y, group_a, group_b = train_data
scaler = StandardScaler()
Xt = scaler.fit_transform(X)
model = LogisticRegression()
model.fit(Xt, y, sample_weight=sw) # do not forget to add sample_weight param when fitting!

# test
X, y, group_a, group_b = test_data
Xt = scaler.transform(X)
y_pred = model.predict(Xt)
y_proba = model.predict_proba(Xt)
y_score = y_proba[:,1]
y_true = y

In [16]:
# efficacy
metrics_dataframe(y_pred, y_true)

,Value
Metric,
Accuracy,0.843484
Balanced accuracy,0.745096
Precision,0.729050
Recall,0.555556
F1-Score,0.630587


In [17]:
# bias
classification_bias_metrics(group_a, group_b, y_pred, y_true, metric_type='both')

,Value,Reference
Metric,,
Statistical Parity,-0.096208,0
Disparate Impact,0.553714,1
Four Fifths Rule,0.553714,1
Cohen D,-0.250423,0
Equality of Opportunity Difference,0.165055,0
False Positive Rate Difference,-0.030316,0
Average Odds Difference,0.067370,0
Accuracy Difference,0.121556,0


The results obtained are better in terms of bias, and efficacy didn't decrease too much.

## Learning Fair Representations
References: 

Zemel, Rich, et al. "Learning fair representations."
        International conference on machine learning. PMLR, 2013.

In [18]:
# import
from holisticai.bias.mitigation import LearningFairRepresentation

In [19]:
# initialise
lfr = LearningFairRepresentation(k=14)

In [20]:
# fit lfr object to training data (remember to standard scale train data)
X_train, y_train, group_a, group_b = train_data
scaler1 = StandardScaler()
X_train_t = scaler1.fit_transform(X_train)
lfr.fit(X_train_t, y_train, group_a, group_b)

In [21]:
# transform train
X_train, y_train, group_a_train, group_b_train = train_data
X_train_t = scaler1.fit_transform(X_train)
new_X_train = lfr.transform(X_train_t, group_a_train, group_b_train)

# transform test
X_test, y_test, group_a_test, group_b_test = test_data
X_test_t = scaler1.fit_transform(X_test)
new_X_test = lfr.transform(X_test_t, group_a_test, group_b_test)

In [22]:
# Fit a model with new data (transformed by LFR algorithm)

# train
X, y, group_a, group_b = train_data
X = new_X_train
scaler2 = StandardScaler()
Xt = scaler2.fit_transform(X)
model = LogisticRegression()
model.fit(Xt, y)

# test
X, y, group_a, group_b = test_data
X = new_X_test
Xt = scaler2.transform(X)
y_pred = model.predict(Xt)
y_proba = model.predict_proba(Xt)
y_score = y_proba[:,1]
y_true = y

In [23]:
# efficacy
metrics_dataframe(y_pred, y_true)

,Value
Metric,
Accuracy,0.767223
Balanced accuracy,0.558005
Precision,0.557427
Recall,0.154960
F1-Score,0.242505


In [24]:
# bias
classification_bias_metrics(group_a, group_b, y_pred, y_true, metric_type='both')

,Value,Reference
Metric,,
Statistical Parity,-0.008035,0
Disparate Impact,0.884467,1
Four Fifths Rule,0.884467,1
Cohen D,-0.032175,0
Equality of Opportunity Difference,0.126363,0
False Positive Rate Difference,-0.004301,0
Average Odds Difference,0.061031,0
Accuracy Difference,0.177722,0


This algorithm has very low recall. It seems fairness is acheived at the expense of efficacy in this case. A good example showing one must be aware of all aspects of the results and not just look at accuracy.

## Inprocessing Strategies

## Grid Search Reduction
References: 

Agarwal, Alekh, et al. "A reductions approach to fair classification."
        International Conference on Machine Learning. PMLR, 2018.

Agarwal, Alekh, Miroslav Dudík, and Zhiwei Steven Wu.
        "Fair regression: Quantitative definitions and reduction-based algorithms."
        International Conference on Machine Learning. PMLR, 2019.

In [25]:
# import
from holisticai.bias.mitigation import GridSearchReduction

In [26]:
# data and model
X, y, group_a, group_b = train_data
scaler = StandardScaler()
Xt = scaler.fit_transform(X)
model = LogisticRegression()

In [27]:
# initialize
gsr = GridSearchReduction(constraints="DemographicParity")

# incorporate model in gsr
gsr.transform_estimator(model)

GridSearchReduction(constraints='DemographicParity')

In [28]:
# fit with data
gsr.fit(Xt, y, group_a, group_b)

GridSearchReduction(constraints='DemographicParity')

In [29]:
# predict test
X, y, group_a, group_b = test_data
Xt = scaler.transform(X)
y_pred = gsr.predict(Xt)
y_proba = gsr.predict_proba(Xt)
y_score = y_proba[:,1]
y_true = y

In [30]:
# efficacy
metrics_dataframe(y_pred, y_true)

,Value
Metric,
Accuracy,0.829358
Balanced accuracy,0.706411
Precision,0.723753
Recall,0.469562
F1-Score,0.569584


In [31]:
# bias
classification_bias_metrics(group_a, group_b, y_pred, y_true, metric_type='both')

,Value,Reference
Metric,,
Statistical Parity,-0.005191,0
Disparate Impact,0.967095,1
Four Fifths Rule,0.967095,1
Cohen D,-0.014306,0
Equality of Opportunity Difference,0.341247,0
False Positive Rate Difference,0.032478,0
Average Odds Difference,0.186862,0
Accuracy Difference,0.113938,0


We have solved for equal demographic parity so statistical parity and Disparate Impact are perfect. We lose some recall but overall efficacy is not impacted too much.

## Exponentiated Gradient Reduction
References:

Agarwal, Alekh, et al. "A reductions approach to fair classification."
        International Conference on Machine Learning. PMLR, 2018.



In [32]:
# import
from holisticai.bias.mitigation import ExponentiatedGradientReduction

In [33]:
# data and model
X, y, group_a, group_b = train_data
scaler = StandardScaler()
Xt = scaler.fit_transform(X)
model = LogisticRegression()

In [34]:
# initialize
egr = ExponentiatedGradientReduction()

# incorporate model in gsr
egr.transform_estimator(model)

ExponentiatedGradientReduction(run_linprog_step=(True,))

In [35]:
# fit with data
egr.fit(Xt, y, group_a, group_b)

ExponentiatedGradientReduction(run_linprog_step=(True,))

In [36]:
# predict test
X, y, group_a, group_b = test_data
Xt = scaler.transform(X)
y_pred = egr.predict(Xt)
y_proba = egr.predict_proba(Xt)
y_score = y_proba[:,1]
y_true = y

In [37]:
# efficacy
metrics_dataframe(y_pred, y_true)

,Value
Metric,
Accuracy,0.836319
Balanced accuracy,0.726850
Precision,0.724014
Recall,0.515964
F1-Score,0.602535


In [38]:
# bias
classification_bias_metrics(group_a, group_b, y_pred, y_true, metric_type='both')

,Value,Reference
Metric,,
Statistical Parity,-0.097127,0
Disparate Impact,0.523907,1
Four Fifths Rule,0.523907,1
Cohen D,-0.259684,0
Equality of Opportunity Difference,0.037593,0
False Positive Rate Difference,-0.017864,0
Average Odds Difference,0.009864,0
Accuracy Difference,0.101617,0


Almost perfect equality of outcome. F1 score is a little bit better than grid search reduction.

## Postprocessing Strategies

## Calibrated Equalized Odds mitigation technique
References:

Pleiss, Geoff, et al. "On fairness and calibration."
    Advances in neural information processing systems 30 (2017).



In [39]:
# import postprocessing strategy
from holisticai.bias.mitigation import CalibratedEqualizedOdds

In [40]:
# Implement a Logistic Regression for baseline results

# pipeline
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression()),
    ])

# train
X_train, y_train, group_a_train, group_b_train = train_data
pipeline.fit(X_train, y_train)

# predict train set
y_pred_train = pipeline.predict(X_train)
y_proba_train = pipeline.predict_proba(X_train)

# predict test set
X_test, y_test, group_a_test, group_b_test = test_data
y_pred_test = pipeline.predict(X_test)
y_proba_test = pipeline.predict_proba(X_test)

In [41]:
# initialize object
ceo = CalibratedEqualizedOdds(cost_constraint="fnr")

In [42]:
# fit it
ceo.fit(y_train, y_proba_train, group_a_train, group_b_train)

In [43]:
# transform it
d = ceo.transform(y_test, y_proba_test, group_a_test, group_b_test, 0.5)

In [44]:
# new predictions
y_pred = d['y_pred']
y_pred

array([0, 0, 1, ..., 0, 0, 0])

In [45]:
# efficacy
metrics_dataframe(y_pred, y_test)

,Value
Metric,
Accuracy,0.831713
Balanced accuracy,0.708107
Precision,0.734531
Recall,0.469987
F1-Score,0.573209


In [46]:
# bias
classification_bias_metrics(group_a_test, group_b_test, y_pred, y_test, metric_type='both')

,Value,Reference
Metric,,
Statistical Parity,-0.111584,0
Disparate Impact,0.416904,1
Four Fifths Rule,0.416904,1
Cohen D,-0.312611,0
Equality of Opportunity Difference,0.095313,0
False Positive Rate Difference,-0.054714,0
Average Odds Difference,0.020299,0
Accuracy Difference,0.150299,0


It seems this algorithm works best with fnr as an option (on Adult dataset). Both fpr and weighted give bad results.

## Equalized Odds mitigation technique
References:

Pleiss, Geoff, et al. "On fairness and calibration."
    Advances in neural information processing systems 30 (2017).

Hardt, Moritz, Eric Price, and Nati Srebro. "Equality of opportunity in supervised learning."
        Advances in neural information processing systems 29 (2016).

In [47]:
# Import
from holisticai.bias.mitigation import EqualizedOdds

In [48]:
# Implement a Logistic Regression for baseline results

# pipeline
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression()),
    ])

# train
X_train, y_train, group_a_train, group_b_train = train_data
pipeline.fit(X_train, y_train)

# predict train set
y_pred_train = pipeline.predict(X_train)
y_proba_train = pipeline.predict_proba(X_train)

# predict test set
X_test, y_test, group_a_test, group_b_test = test_data
y_pred_test = pipeline.predict(X_test)
y_proba_test = pipeline.predict_proba(X_test)

In [49]:
# initialize
eq = EqualizedOdds(solver='highs', seed=42)

In [50]:
# fit it
eq.fit(y_train, y_pred_train, group_a_train, group_b_train)

In [51]:
# transform
d = eq.transform(y_pred_test, group_a_test, group_b_test)
d

{'y_pred': array([0, 1, 1, ..., 1, 0, 0]),
 'y_score': array([0., 1., 1., ..., 1., 0., 0.])}

In [52]:
# new predictions
y_pred = d['y_pred']
y_pred

array([0, 1, 1, ..., 1, 0, 0])

In [53]:
# efficacy
metrics_dataframe(y_pred, y_test)

,Value
Metric,
Accuracy,0.825980
Balanced accuracy,0.733282
Precision,0.665815
Recall,0.554704
F1-Score,0.605202


In [54]:
# bias metrics
classification_bias_metrics(group_a_test, group_b_test, y_pred, y_test, metric_type='both')

,Value,Reference
Metric,,
Statistical Parity,-0.095813,0
Disparate Impact,0.587965,1
Four Fifths Rule,0.587965,1
Cohen D,-0.240930,0
Equality of Opportunity Difference,0.020744,0
False Positive Rate Difference,-0.010227,0
Average Odds Difference,0.005258,0
Accuracy Difference,0.080495,0


Each algorithm will mitigate in slightly different ways. Notice the average odds difference is very low.

## Reject Option Classification
References:

Kamiran, Faisal, Asim Karim, and Xiangliang Zhang. "Decision theory for discrimination-aware classification."
    2012 IEEE 12th International Conference on Data Mining. IEEE, 2012.

In [55]:
# import
from holisticai.bias.mitigation import RejectOptionClassification

In [56]:
# Implement a Logistic Regression for baseline results

# pipeline
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression()),
    ])

# train
X_train, y_train, group_a_train, group_b_train = train_data
pipeline.fit(X_train, y_train)

# predict train set
y_pred_train = pipeline.predict(X_train)
y_proba_train = pipeline.predict_proba(X_train)

# predict test set
X_test, y_test, group_a_test, group_b_test = test_data
y_pred_test = pipeline.predict(X_test)
y_proba_test = pipeline.predict_proba(X_test)

In [57]:
# initialize
roc = RejectOptionClassification(metric_name="Statistical parity difference")

In [58]:
# fit it
roc.fit(y_train, y_proba_train, group_a_train, group_b_train)

In [59]:
# transform it
d = roc.transform(y_test, y_proba_test, group_a_test, group_b_test)
d

{'y_pred': array([0, 1, 1, ..., 1, 1, 0]),
 'y_score': array([0.09544274, 0.84811759, 0.61947223, ..., 0.52319612, 0.14669624,
        0.14180466])}

In [60]:
# new predictions
y_pred = d['y_pred']
y_pred

array([0, 1, 1, ..., 1, 1, 0])

In [61]:
# efficacy
metrics_dataframe(y_pred, y_test)

,Value
Metric,
Accuracy,0.779609
Balanced accuracy,0.788730
Precision,0.527283
Recall,0.806301
F1-Score,0.637603


In [62]:
# bias metrics
classification_bias_metrics(group_a_test, group_b_test, y_pred, y_test, metric_type='both')

,Value,Reference
Metric,,
Statistical Parity,-0.059864,0
Disparate Impact,0.845638,1
Four Fifths Rule,0.845638,1
Cohen D,-0.124368,0
Equality of Opportunity Difference,0.148833,0
False Positive Rate Difference,0.038114,0
Average Odds Difference,0.093473,0
Accuracy Difference,-0.016622,0


Great results! We lose some accuracy, but recall increases.